In [1]:
import numpy as np
import matplotlib.pyplot as plt

import pickle
import os
from ANEMO import read_edf
from ANEMO import ANEMO

from scipy import stats
import scipy as sp

%matplotlib inline

In [2]:
sujet = ['AM','BMC','CS','DC','FM','IP','LB','OP','RS','SR','TN','YK']
time = ['2017-10-23_100057','2017-09-26_095637', '2017-10-03_134421','2017-09-27_161040',
        '2017-10-03_143803','2017-09-28_115250', '2017-09-20_151043','2017-10-26_121823',
        '2017-11-08_094717','2017-11-16_153313', '2017-11-08_150410','2017-11-17_172706']

In [3]:
file = os.path.join('parametre', 'Delete_list_trials_velocity_fct.pkl')
with open(file, 'rb') as fichier :
    Delete_list_trials = pickle.load(fichier, encoding='latin1')

file = os.path.join('parametre', 'Delete_list_Bad_trials_velocity.pkl')
#file = os.path.join('parametre', 'Delete_list_Bad_trials_position.pkl')
with open(file, 'rb') as fichier :
    Bad_Fit = pickle.load(fichier, encoding='latin1')

In [4]:
cool = {}
bad = {}

list_keys_param_fit = ['start_anti', 'tau', 'a_anti', 'latency', 'maxi']
for dic in [cool, bad] :
    dic['nb_nan_GAP'] = []
    dic['nb_nan_anti'] = []
    dic['nb_nan_rise'] = []
    
    dic['residu_GAP'] = []
    dic['residu_anti'] = []
    dic['residu_rise'] = []
    
    dic['comp_fit_pos'] = []
    dic['larg_sacc'] = []
    dic['integrale_pos'] = []
    dic['nb_sacc'] = []
    for key in list_keys_param_fit :
        dic[key]=[]


a=0
for x in range(len(sujet)) :
    print(sujet[x], end=' ')
    
    #--------------------------------------------------------------------------------------
    file = os.path.join('data', 'enregistrement_' + sujet[x] + '_' + time[x] + '.pkl')
    with open(file, 'rb') as fichier : param_exp = pickle.load(fichier, encoding='latin1')
    
    resultats = os.path.join('data', 'enregistrement_' + sujet[x] + '_' + time[x] + '.asc')
    data = read_edf(resultats, 'TRIALID')

    file = os.path.join('parametre/param_Fit_'+ sujet[x] +'_fct_velocity_2_step_False_whitening.pkl')
    with open(file, 'rb') as fichier : param_fit = pickle.load(fichier, encoding='latin1')

    file = os.path.join('parametre/goodness_of_fit/goodness_of_fit_'+ sujet[x] +'.pkl')
    with open(file, 'rb') as fichier : goodness_of_fit = pickle.load(fichier, encoding='latin1')
    #--------------------------------------------------------------------------------------
    
    
    A = ANEMO(param_exp)
    Plot = A.Plot(param_exp)
    
    N_trials, N_blocks = param_exp['N_trials'], param_exp['N_blocks']

    for block in range(N_blocks) :
        for trial in range(N_trials) :

            if trial in Delete_list_trials[sujet[x]][block] : dic = bad
            elif trial in Bad_Fit[sujet[x]][block] : dic = bad
            else : dic = cool


                
                
            trial_data = trial + N_trials*block
            arg = A.arg(data[trial_data], trial=trial, block=block)
            velocity_NAN = A.velocity_NAN(**arg)
            
            onset = arg.TargetOn-arg.t_0
            xx = np.arange(len(arg.trackertime))

            start_anti = int(param_fit['start_anti'][block][trial] + onset)
            a_anti = param_fit['a_anti'][block][trial]

            latency = int(param_fit['latency'][block][trial] + onset)
            tau = param_fit['tau'][block][trial]
            maxi = param_fit['maxi'][block][trial]
            start, stop = start_anti, -1
            
            #------------------------------------------------
            # Param_Fit
            #------------------------------------------------
            for key in list_keys_param_fit :
                if key=='a_anti' :
                    dic[key].append((param_fit[key][block][trial])*arg.dir_target)
                else :
                    dic[key].append(param_fit[key][block][trial])
            
            #--------------------------------------------------------------------------------------
            
            
            #------------------------------------------------
            # Nb_NaN
            #------------------------------------------------
            nb_nan = [x for x in velocity_NAN[arg.StimulusOf-arg.t_0:arg.TargetOn-arg.t_0] if str(x)=='nan']
            dic['nb_nan_GAP'].append(len(nb_nan)/len(velocity_NAN[arg.StimulusOf-arg.t_0:arg.TargetOn-arg.t_0]))
            
            nb_nan = [x for x in velocity_NAN[start_anti:latency] if str(x)=='nan']
            dic['nb_nan_anti'].append(len(nb_nan)/len(velocity_NAN[start_anti:latency]))
            
            nb_nan = [x for x in velocity_NAN[latency:len(velocity_NAN)-280] if str(x)=='nan']
            dic['nb_nan_rise'].append(len(nb_nan)/len(velocity_NAN[latency:len(velocity_NAN)-280]))
            #--------------------------------------------------------------------------------------
            
            
            #------------------------------------------------
            # Residu
            #------------------------------------------------
            t1 , res = 0, np.zeros(len(velocity_NAN)-280)
            for t in range(len(velocity_NAN)-280) :
                if np.isnan(velocity_NAN[t]) : res[t] = np.nan
                else : res[t] = abs(goodness_of_fit['residual'][block][trial][t1]) ; t1 = t1+1

            if np.isnan(np.nanmean(res[arg.StimulusOf-arg.t_0:arg.TargetOn-arg.t_0])) : dic['residu_GAP'].append(0)
            else : dic['residu_GAP'].append(np.nanmean(res[arg.StimulusOf-arg.t_0:arg.TargetOn-arg.t_0]))
                
            if np.isnan(np.nanmean(res[start_anti:latency])) : dic['residu_anti'].append(0)
            else : dic['residu_anti'].append(np.nanmean(res[start_anti:latency]))
                
            if np.isnan(np.nanmean(res[latency:])) : dic['residu_rise'].append(0)
            else : dic['residu_rise'].append(np.nanmean(res[latency:]))
            #--------------------------------------------------------------------------------------
            
            after_sacc, before_sacc = 15, 5
            sacc = np.zeros(len(arg.trackertime))
            i=0
            for s in range(len(arg.saccades)):
                for j in range(3) : sacc[i+j] = arg.saccades[s][j]
                i = i+3                    

            fit = A.Equation.fct_position(xx, arg.data_x, sacc, len(arg.saccades), arg.dir_target,
                                          start_anti , a_anti, latency, tau, maxi, arg.t_0, arg.px_per_deg,
                                          before_sacc, after_sacc, do_whitening=False)
            d_x = A.data_deg(arg.data_x, arg.StimulusOf, arg.t_0, arg.saccades, before_sacc, after_sacc)
            
            fit = fit[:-280]
            d_x = d_x[:-280]
            #--------------------------------------------------------------------------------------
            
            # Méthode des trapèzes
            d_x_2 = np.ma.masked_array(d_x[start_anti:], mask=np.isnan(fit[start_anti:])).compressed()
            dic['integrale_pos'].append(sp.trapz(d_x_2)*arg.dir_target)
            
            #--------------------------------------------------------------------------------------
            larg_sacc, nb_sacc = 0, 0
            for s in range(len(arg.saccades)) :
                sacc = arg.saccades[s]
                
                start_sacc = int(sacc[0]-arg.t_0)-int(before_sacc)
                stop_sacc = int(sacc[1]-arg.t_0)+int(after_sacc)
                
                if stop_sacc+1 <= len(arg.data_x[:-280]) :
                    
                    if s < len(arg.saccades)-1 :
                        start_s2 = int(arg.saccades[s+1][0]-arg.t_0)-int(before_sacc)-1
                    
                        d_x[stop_sacc:] = d_x[stop_sacc:]-np.mean(d_x[stop_sacc:start_s2])
                        fit[stop_sacc:] = fit[stop_sacc:]-np.mean(fit[stop_sacc:start_s2])
                    else :
                        d_x[stop_sacc:] = d_x[stop_sacc:]-np.mean(d_x[stop_sacc:])
                        fit[stop_sacc:] = fit[stop_sacc:]-np.mean(fit[stop_sacc:])

                    if sacc[0] >= arg.StimulusOf :
                        nb_sacc = nb_sacc + 1
                        larg_sacc = larg_sacc+abs((arg.data_x[stop_sacc]-arg.data_x[start_sacc-1])/arg.px_per_deg)
            #--------------------------------------------------------------------------------------
            
            dic['nb_sacc'].append(nb_sacc)
            dic['larg_sacc'].append(larg_sacc)
            
            #--------------------------------------------------------------------------------------
            d_x_ss_nan = np.ma.masked_array(d_x[start:stop], mask=np.isnan(fit[start:stop])).compressed()
            fit_ss_nan = np.ma.masked_array(fit[start:stop], mask=np.isnan(fit[start:stop])).compressed()
            #--------------------------------------------------------------------------------------
        
            dic['comp_fit_pos'].append(np.nanmean((d_x_ss_nan-fit_ss_nan)**2))

AM BMC CS DC FM IP LB 

/home/drone/.local/lib/python3.5/site-packages/ipykernel_launcher.py:105: RuntimeWarning: Mean of empty slice


OP 

/home/drone/.local/lib/python3.5/site-packages/ipykernel_launcher.py:108: RuntimeWarning: Mean of empty slice


RS SR 

/usr/local/lib/python3.5/dist-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.5/dist-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/drone/.local/lib/python3.5/site-packages/ipykernel_launcher.py:167: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.5/dist-packages/ANEMO-0.0.1-py3.5.egg/ANEMO/ANEMO.py:438: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.5/dist-packages/ANEMO-0.0.1-py3.5.egg/ANEMO/ANEMO.py:438: RuntimeWarning: invalid value encountered in true_divide


TN YK 

In [5]:
print(len(cool['comp_fit_pos']))
print(len(bad['comp_fit_pos']))

6585
615


In [6]:
for k in cool.keys() :
    if len(k)<7 : t= ' '*3
    else : t=''
    if abs(np.nanmean(cool[k]))>abs(np.nanmean(bad[k])) :
        g = 'cool'
        p = (np.nanmean(bad[k])/np.nanmean(cool[k]))*100
    else:
        g='bad '
        p = (np.nanmean(cool[k])/np.nanmean(bad[k]))*100
    print(k, t, '\t--\tcool: %0.3f \tbad: %0.3f'%(np.nanmean(cool[k]), np.nanmean(bad[k])),
         '\t--\t', g, '\t%0.1f'%p)

larg_sacc  	--	cool: 3.861 	bad: 5.655 	--	 bad  	68.3
residu_anti  	--	cool: 3.113 	bad: 3.741 	--	 bad  	83.2
nb_nan_anti  	--	cool: 0.025 	bad: 0.135 	--	 bad  	18.2
residu_rise  	--	cool: 5.214 	bad: 6.460 	--	 bad  	80.7
start_anti  	--	cool: -95.556 	bad: -128.620 	--	 bad  	74.3
residu_GAP  	--	cool: 2.992 	bad: 3.474 	--	 bad  	86.1
integrale_pos  	--	cool: 1920.637 	bad: 1756.781 	--	 cool 	91.5
nb_nan_GAP  	--	cool: 0.042 	bad: 0.155 	--	 bad  	27.4
tau     	--	cool: 45.928 	bad: 48.064 	--	 bad  	95.6
nb_nan_rise  	--	cool: 0.178 	bad: 0.281 	--	 bad  	63.3
latency  	--	cool: 175.207 	bad: 171.497 	--	 cool 	97.9
nb_sacc  	--	cool: 1.868 	bad: 2.185 	--	 bad  	85.5
a_anti     	--	cool: 10.244 	bad: 0.108 	--	 cool 	1.1
comp_fit_pos  	--	cool: 0.008 	bad: 0.027 	--	 bad  	31.5
maxi     	--	cool: 12.207 	bad: 11.295 	--	 cool 	92.5
